This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/lyapunov.html).  I recommend having both windows open, side-by-side!


In [ ]:
import numpy as np
from pydrake.all import (Jacobian, MathematicalProgram, RegionOfAttraction,
                         Solve, SymbolicVectorSystem, Variable, Variables)

# Region of attraction for the one-dimensional cubic system

In [ ]:
def sos_roa():
    prog = MathematicalProgram()
    x = prog.NewIndeterminates(1, "x")

    # Define the dynamics and Lyapunov function.
    f = -x + x**3
    V = x.dot(x)
    Vdot = Jacobian([V], x).dot(f)[0]

    # Define the Lagrange multiplier.
    lambda_ = prog.NewSosPolynomial(Variables(x), 2)[0].ToExpression()

    prog.AddSosConstraint(-Vdot - lambda_*(1 - V))

    result = Solve(prog)

    assert result.is_success(), "Optimization failed"

    print("Verified that " + str(V) + " < 1 is in the region of attraction.")

sos_roa()

In [ ]:
def sos_roa2():
    prog = MathematicalProgram()
    x = prog.NewIndeterminates(1, "x")
    rho = prog.NewContinuousVariables(1, "rho")[0]

    # Define the dynamics and Lyapunov function.
    f = -x + x**3
    V = x.dot(x)
    Vdot = Jacobian([V], x).dot(f)[0]

    # Define the Lagrange multiplier.
    lambda_ = prog.NewContinuousVariables(1, "lambda")[0]

    prog.AddSosConstraint((V - rho) * x.dot(x) - lambda_ * Vdot)
    prog.AddLinearCost(-rho)

    result = Solve(prog)

    assert result.is_success()

    print("Verified that " + str(V) + " < " + str(result.GetSolution(rho)) +
        " is in the region of attraction.")

    assert np.fabs(result.GetSolution(rho) - 1) < 1e-5

sos_roa2()

# Region of Attraction codes in Drake

In [ ]:
def roa():
    x = Variable("x")
    sys = SymbolicVectorSystem(state=[x], dynamics=[-x+x**3])
    context = sys.CreateDefaultContext()
    V = RegionOfAttraction(system=sys, context=context)

    print("Verified that " + str(V) + " < 1 is in the region of attraction.")

roa()